Example illustrating the case of sending an empty list to a ListPlaceNode destination.

In [ ]:
from typing import Optional
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
def generate_list_of_integers(n: int) -> list[int]:
    return [100 + x for x in range(n)]

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode(name="Input Number", type=int, tokens=[1, 2, 0, 3]),

    ArgumentEdgeToTransition("Input Number", "Generate List of Integers", "n"),
    FunctionTransitionNode(
        name="Generate List of Integers",
        function=generate_list_of_integers,
    ),
    ReturnedEdgeFromTransition("Generate List of Integers", "Integer Lists"),

    ListPlaceNode(name="Integer Lists", type=list[int]),

    ArgumentEdgeToTransition("Integer Lists", "Pass Through List", argument="lst"),
    FunctionTransitionNode(
        name="Pass Through List",
        function=lambda lst: lst
    ),
    ReturnedEdgeFromTransition("Pass Through List", "Final Output"),

    ListPlaceNode(name="Final Output", type=int),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
display(SimpleGraphvizVisualization.graph(executable_pydigraph))

In [ ]:
async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()